In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
x_test =  pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')
df = pd.concat([x_test, y_test], axis=1)

# feature engineer

In [3]:
df['have_prev_address'] = np.where(df['prev_address_months_count']<0, 0, 1)

df['have_initial_bal'] = np.where(df['intended_balcon_amount']<=0, 0, 1)

In [4]:
df['bank_integration'] = df['bank_months_count'] + df['has_other_cards']*df['bank_months_count']

df['income_credit_limit_ratio'] = df['income'] / df['proposed_credit_limit']

# standardisation

In [5]:
cluster_features = [
            'fraud_bool',
            'income',
            'name_email_similarity',
            'current_address_months_count',
            'customer_age',
            'days_since_request',
            'payment_type',
            'zip_count_4w',
            'bank_branch_count_8w',
            'date_of_birth_distinct_emails_4w',
            'employment_status',
            'credit_risk_score',
            'email_is_free',
            'housing_status',
            'phone_home_valid',
            'phone_mobile_valid',
            'bank_months_count', 
            'has_other_cards',
            'proposed_credit_limit',
            'foreign_request',
            'device_os',
            'device_distinct_emails_8w',
            'bank_integration',
            'income_credit_limit_ratio',
            'have_prev_address',
            'have_initial_bal'
                ]

In [6]:
numerical_features = [
#             'fraud_bool',
            'income',
            'name_email_similarity',
            'current_address_months_count',
            'customer_age',
            'days_since_request',
#             'payment_type',
            'zip_count_4w',
            'bank_branch_count_8w',
            'date_of_birth_distinct_emails_4w',
#             'employment_status',
            'credit_risk_score',
#             'email_is_free',
#             'housing_status',
#             'phone_home_valid',
#             'phone_mobile_valid',
            'bank_months_count', 
#             'has_other_cards',
            'proposed_credit_limit',
#             'foreign_request',
#             'device_os',
            'device_distinct_emails_8w',
            'bank_integration',
            'income_credit_limit_ratio',
#             'have_prev_address',
#             'have_initial_bal'
#             'have_bank_months_count',
                ]

In [7]:
standardisation_dict = pickle.load(open('standardisation_dict.pkl', 'rb'))

In [8]:
df_scaled = df[cluster_features]

for col in numerical_features:
    mean = standardisation_dict['mean'][col] 
    std = standardisation_dict['std'][col]
    df_scaled[col] = (df_scaled[col] - mean) / std 

C:\Users\marcu\AppData\Local\Temp\ipykernel_14504\757233552.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scaled[col] = (df_scaled[col] - mean) / std


# predict kmeans_label

In [9]:
kmeans_model = pickle.load(open('kmeans_model.pkl', 'rb'))

In [10]:
categorical_features = [
#             'fraud_bool',
#             'income',
#             'name_email_similarity',
#             'current_address_months_count',
#             'customer_age',
#             'days_since_request',
            'payment_type',
#             'zip_count_4w',
#             'bank_branch_count_8w',
#             'date_of_birth_distinct_emails_4w',
            'employment_status',
#             'credit_risk_score',
            'email_is_free',
            'housing_status',
            'phone_home_valid',
            'phone_mobile_valid',
#             'bank_months_count', 
            'has_other_cards',
#             'proposed_credit_limit',
            'foreign_request',
            'device_os',
#             'device_distinct_emails_8w',
#             'bank_integration',
#             'income_credit_limit_ratio',
            'have_prev_address',
            'have_initial_bal',
#             'have_bank_months_count'
                ]

In [11]:
df_cluster = pd.get_dummies(df_scaled, columns=categorical_features)
df_cluster.head()

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,zip_count_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,...,foreign_request_1,device_os_linux,device_os_macintosh,device_os_other,device_os_windows,device_os_x11,have_prev_address_0,have_prev_address_1,have_initial_bal_0,have_initial_bal_1
0,0,-0.969891,-0.455141,-0.275682,0.410756,-0.187634,-0.735028,-0.281376,-1.018837,0.016546,...,0,0,0,0,1,0,1,0,1,0
1,0,1.097434,-0.969974,0.749508,0.410756,-0.183292,-0.855678,-0.368258,-1.215585,1.182879,...,0,0,0,0,1,0,1,0,1,0
2,0,1.097434,-1.318128,-0.681252,-1.192116,-0.181924,-0.623352,-0.377169,1.735630,0.898077,...,0,0,0,0,1,0,1,0,0,1
3,1,1.097434,0.790311,-0.467201,1.212192,-0.181309,0.169352,3.989209,-0.231847,-0.580182,...,0,0,0,0,1,0,1,0,1,0
4,0,-0.969891,-0.180450,-0.895303,-1.192116,1.473386,0.658933,-0.388307,1.735630,-1.326092,...,0,1,0,0,0,0,0,1,1,0


In [12]:
labels = kmeans_model.predict(df_cluster)

In [13]:
df_scaled['kmeans_label'] = labels
df_scaled.head()

C:\Users\marcu\AppData\Local\Temp\ipykernel_14504\3650804785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scaled['kmeans_label'] = labels


,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,payment_type,zip_count_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,...,has_other_cards,proposed_credit_limit,foreign_request,device_os,device_distinct_emails_8w,bank_integration,income_credit_limit_ratio,have_prev_address,have_initial_bal,kmeans_label
0,0,-0.969891,-0.455141,-0.275682,0.410756,-0.187634,AB,-0.735028,-0.281376,-1.018837,...,0,-0.128522,0,windows,-0.147885,0.129002,-0.846761,0,0,1
1,0,1.097434,-0.969974,0.749508,0.410756,-0.183292,AB,-0.855678,-0.368258,-1.215585,...,1,-0.128522,0,windows,-0.147885,-0.686660,-0.054951,0,0,3
2,0,1.097434,-1.318128,-0.681252,-1.192116,-0.181924,AD,-0.623352,-0.377169,1.735630,...,0,1.762963,0,windows,-0.147885,0.944664,-0.846761,0,1,3
3,1,1.097434,0.790311,-0.467201,1.212192,-0.181309,AD,0.169352,3.989209,-0.231847,...,0,-0.695968,0,windows,-0.147885,-0.749403,1.726620,0,0,4
4,0,-0.969891,-0.180450,-0.895303,-1.192116,1.473386,AC,0.658933,-0.388307,1.735630,...,0,-0.695968,0,linux,-0.147885,-0.874890,-0.252904,1,0,1


# use labels for new column

In [14]:
# df_cluster = pd.DataFrame({
#     'housing_status': ['BA', 'AA', 'CA', 'BA', 'BA'],
#     'kmeans_label': [0, 1, 0, 1, 1],
#     'fraud_bool': [1, 0, 1, 0, 1],
#     'device_os': ['windows', 'windows', 'android', 'windows', 'android'],
#     'date_of_birth_distinct_emails_4w': [0.5, -1.5, 0.0, 2.0, -2.0],
#     'credit_risk_score': [0.3, 2.5, 0.2, 0.6, 1.7],
#     'income': [0.5, 2.0, 1.5, 3.0, 2.5],
#     'payment_type': ['AC', 'CC', 'DC', 'AC', 'DC'],
#     'bank_branch_count_8w': [-0.5, 0.1, -0.4, 0.2, -0.1],
#     'customer_age': [25, 30, 35, 40, 45],
#     'proposed_credit_limit': [3.0, 2.5, 2.0, 2.6, 3.5],
#     'phone_home_valid': [1, 0, 1, 0, 1],
#     'has_other_cards': [0, 1, 0, 1, 0],
#     'email_is_free': [1, 0, 1, 0, 1],
#     'bank_months_count': [-0.8, -1.0, -0.5, -0.9, -1.1],
#     'kmeans_label': [0, 2, 1, 3, 3]
# })
df_cluster = df_scaled

In [15]:
condition_list = [
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['proposed_credit_limit']>=2.474),
        (df_cluster['housing_status']=='BA') & (df_cluster['kmeans_label']==3),
        (df_cluster['kmeans_label']==3) & (df_cluster['housing_status']=='BA'),
        (df_cluster['kmeans_label']==3) & (df_cluster['credit_risk_score']>=0.792),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) &(df_cluster['housing_status']=='BA') &(df_cluster['bank_months_count']<=-0.91),
        (df_cluster['device_os']=='windows') &(df_cluster['kmeans_label'].isin([1])) & (df_cluster['housing_status']=='BA') & (df_cluster['bank_months_count']<=-0.91),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) &(df_cluster['housing_status']=='BA') & (df_cluster['payment_type'].isin(['AC'])),
        (df_cluster['device_os']=='windows') &(df_cluster['kmeans_label'].isin([1])) &(df_cluster['housing_status']=='BA') &(df_cluster['income']>=1),
        (df_cluster['device_os'] == 'windows') &   (df_cluster['kmeans_label'].isin([1])) & (df_cluster['housing_status'].isin(['BA'])) &   (df_cluster['phone_home_valid'].isin([0])),
        (df_cluster['kmeans_label'].isin([1])) & (df_cluster['phone_home_valid']==0) & (df_cluster['date_of_birth_distinct_emails_4w']<=-1.404),
        (df_cluster['kmeans_label']==3) & (df_cluster['date_of_birth_distinct_emails_4w']< -1.4),
        (df_cluster['device_os']=='windows') &(df_cluster['kmeans_label'].isin([1])) & (df_cluster['housing_status']=='BA') &(df_cluster['customer_age']> 1),
        (df_cluster['device_os'] == 'windows') &  (df_cluster['kmeans_label'].isin([1])) & (df_cluster['housing_status'].isin(['BA'])) & (df_cluster['has_other_cards'].isin([0])),
        (df_cluster['kmeans_label']==3) & (df_cluster['device_os']=='windows'),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==3),
        (df_cluster['kmeans_label'].isin([1])) & (df_cluster['has_other_cards']==0) & (df_cluster['credit_risk_score']>=2.2),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['date_of_birth_distinct_emails_4w'] < -1.215),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['credit_risk_score']>=1.6),
        (df_cluster['device_os']=='windows') &(df_cluster['kmeans_label'].isin([1])) &(df_cluster['housing_status']=='BA') & (df_cluster['date_of_birth_distinct_emails_4w']<=-0.035),
        (df_cluster['device_os']=='windows') &(df_cluster['kmeans_label'].isin([1])) & (df_cluster['housing_status']=='BA') & (df_cluster['bank_branch_count_8w']<=0.19),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['income']>1),
        (df_cluster['kmeans_label'].isin([1])) & (df_cluster['has_other_cards']==0) & (df_cluster['date_of_birth_distinct_emails_4w']<=-1.404),
        (df_cluster['kmeans_label']==3) & (df_cluster['income']>1),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['housing_status']=='BA'),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['payment_type'].isin(['AC'])),
       (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['bank_months_count'] <= -0.91),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['bank_branch_count_8w']<-0.386),
        (df_cluster['credit_risk_score']>=2.2),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['phone_home_valid']==0),
        (df_cluster['kmeans_label'].isin([1])) & (df_cluster['phone_home_valid']==0) & (df_cluster['device_os']=='windows'),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])) & (df_cluster['email_is_free']==1),
        (df_cluster['kmeans_label'].isin([1])) & (df_cluster['phone_home_valid']==0) & (df_cluster['housing_status']=='BA'),
       (df_cluster['date_of_birth_distinct_emails_4w']<=-1.404) & (df_cluster['kmeans_label']==1),
        (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label'].isin([1])),
        (df_cluster['kmeans_label']==3),
       (df_cluster['phone_home_valid']==0) & (df_cluster['kmeans_label'].isin([1])),
        (df_cluster['kmeans_label'].isin([1])) & (df_cluster['has_other_cards']==0),
        (df_cluster['kmeans_label']==4) & (df_cluster['housing_status']=='BA'),
       (df_cluster['kmeans_label']==1),
       (df_cluster['device_os']=='windows') ,
        (df_cluster['kmeans_label']==2) & (df_cluster['housing_status']=='BA'),
        (df_cluster['kmeans_label']==0) & (df_cluster['housing_status']=='BA'),
        (df_cluster['kmeans_label']==4),
        (df_cluster['kmeans_label']==2),
        (df_cluster['kmeans_label']==0)
                   ]

prob_list = [0.802345059,
                0.731907895,
                0.731907895,
                0.72826087,
                0.688747731,
                0.688747731,
                0.68867083,
                0.675824176,
                0.652659812,
                0.636968085,
                0.631578947,
                0.622473246,
                0.621749409,
                0.621428571,
                0.621428571,
                0.620403321,
                0.60994561,
                0.603804348,
                0.600875593,
                0.594287569,
                0.582660433,
                0.581125828,
                0.581081081,
                0.579007743,
                0.576676385,
                0.572434018,
                0.571984436,
                0.57054126,
                0.550396376,
                0.550396376,
                0.530743347,
                0.528811087,
                0.52045607,
                0.461364427,
                0.439457203,
                0.387566845,
                0.365830916,
                0.361952101,
                0.313652839,
                0.307911111,
                0.30556813,
                0.257042254,
                0.134435711,
                0.10993009,
                0.066630107
                ]

In [16]:
df_cluster['kmeans_prob'] = np.select(condition_list, prob_list, 0)

C:\Users\marcu\AppData\Local\Temp\ipykernel_14504\1568432720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster['kmeans_prob'] = np.select(condition_list, prob_list, 0)


In [17]:
df_cluster

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,payment_type,zip_count_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,...,proposed_credit_limit,foreign_request,device_os,device_distinct_emails_8w,bank_integration,income_credit_limit_ratio,have_prev_address,have_initial_bal,kmeans_label,kmeans_prob
0,0,-0.969891,-0.455141,-0.275682,0.410756,-0.187634,AB,-0.735028,-0.281376,-1.018837,...,-0.128522,0,windows,-0.147885,0.129002,-0.846761,0,0,1,0.652660
1,0,1.097434,-0.969974,0.749508,0.410756,-0.183292,AB,-0.855678,-0.368258,-1.215585,...,-0.128522,0,windows,-0.147885,-0.686660,-0.054951,0,0,3,0.731908
2,0,1.097434,-1.318128,-0.681252,-1.192116,-0.181924,AD,-0.623352,-0.377169,1.735630,...,1.762963,0,windows,-0.147885,0.944664,-0.846761,0,1,3,0.728261
3,1,1.097434,0.790311,-0.467201,1.212192,-0.181309,AD,0.169352,3.989209,-0.231847,...,-0.695968,0,windows,-0.147885,-0.749403,1.726620,0,0,4,0.307911
4,0,-0.969891,-0.180450,-0.895303,-1.192116,1.473386,AC,0.658933,-0.388307,1.735630,...,-0.695968,0,linux,-0.147885,-0.874890,-0.252904,1,0,1,0.387567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13189,1,1.097434,1.230358,1.312800,2.013627,-0.182386,AC,0.836419,-0.386080,-1.412333,...,-0.109607,0,windows,-0.147885,1.070150,-0.078240,0,0,3,0.731908
13190,1,1.097434,-0.973322,1.695838,2.815063,-0.183920,AB,-0.691155,-0.368258,-0.625342,...,-0.695968,0,windows,-0.147885,0.944664,1.726620,0,0,2,0.307911
13191,1,0.408326,1.212468,-0.106695,0.410756,-0.183545,AD,-0.143741,1.358244,-0.035099,...,1.762963,0,windows,-0.147885,-0.686660,-0.934740,0,1,3,0.731908
13192,0,1.097434,-1.314917,3.059003,0.410756,-0.181954,AB,0.220204,1.572107,-1.018837,...,1.762963,0,windows,-0.147885,1.007407,-0.846761,0,0,3,0.731908
